In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import optuna

from category_encoders import WOEEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Downloading data

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e3/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e3/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e3/sample_submission.csv')
addition_data = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

train_df['is_generated'] = 1
test_df['is_generated'] = 1
addition_data['is_generated'] = 0

In [ ]:
train_df = train_df.drop(527).drop(1398).reset_index(drop=True)

In [ ]:
train_df = train_df.drop('id', axis=1)
train_df

In [ ]:
# addition_data.EmployeeCount.hist()

No missing values in dataset

In [ ]:
train_df.isna().any()

In [ ]:
addition_data['Attrition'] = (addition_data['Attrition'] == 'Yes').astype(int)
addition_data

In [ ]:
addition_data.Attrition.hist()

In [ ]:
addition_data.isna().any()

In [ ]:
train_df = pd.concat([train_df, addition_data],axis=0, ignore_index=True)
train_df = train_df.drop('EmployeeNumber', axis=1)
train_df

In [ ]:
train_df.isna().any()

In [ ]:
def is_young(x):
    if x <=25:
        return 1
    else:
        return 0
    
def young_and_low_daily_rate(x):
    if x['Age'] <= 25 & x['DailyRate'] < 500:
        return 1
    else:
        return 0
    
def overtime_satisfaction(x):
        if x['OverTime'] == 'Yes':
            return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 0.05) * x['JobSatisfaction'])/x['Age']
        else:
            return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 1.05) * x['JobSatisfaction'])/x['Age']
        
# def calculate_experience_ratios(df):
#     '''
#     Calculates multiple experience ratios...
#     '''
#     df['Equivalent_Salary'] = df['MonthlyIncome'] / (df['TotalWorkingYears'] + 1)
#     df['Working_Years'] = df['YearsAtCompany'] / (df['TotalWorkingYears'] + 1)
#     df['Avg_Years_Per_Company'] = df['TotalWorkingYears'] / (df['NumCompaniesWorked'] + 1)
#     df['Education_To_Salary'] = df['MonthlyIncome'] / (df['Education'] + 1)
#     df['Manager_Familiarity'] = df['YearsWithCurrManager'] / (df['YearsAtCompany'] + 1)
#     df['Performance_Fairness'] = df['PercentSalaryHike'] / df['PerformanceRating']
#     return df

In [ ]:
df = pd.concat([train_df, test_df], axis=0)
df = df.drop(["EmployeeCount", "Over18", "StandardHours"], axis=1)

df['is_young'] = df['Age'].apply(lambda x: is_young(x))
df['young_and_underpaid'] = df.apply(lambda x: young_and_low_daily_rate(x), axis = 1)
df['worklife_stock'] = df.apply(lambda x: x['WorkLifeBalance'] + x['StockOptionLevel'], axis = 1)

df['income_satisfaction'] = df.apply(lambda x: x['JobSatisfaction'] * x['MonthlyIncome'], axis = 1)
df['income_level_environ_job_sat'] = df.apply(lambda x: x['EnvironmentSatisfaction']*x['JobSatisfaction'] * (x['MonthlyIncome']/x['JobLevel']), axis = 1)
df['overtime_stock'] = df.apply(lambda x: overtime_satisfaction(x), axis = 1)

# df = calculate_experience_ratios(df)

# df = pd.get_dummies(df)
df

In [ ]:
features = ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'is_generated', 'id', 'is_young',
       'young_and_underpaid', 'worklife_stock', 'income_satisfaction',
        'income_level_environ_job_sat', 'overtime_stock']
cat_features = ['BusinessTravel', 'Department','Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
               'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus','NumCompaniesWorked', 'OverTime', 
               'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 
                'WorkLifeBalance', 'YearsAtCompany','is_young', 'young_and_underpaid']

Woe encoder from @faelk8 notebook https://www.kaggle.com/code/faelk8/catboost/notebook

In [ ]:
woe = WOEEncoder(drop_invariant=True, randomized = True)
for col in cat_features:
    df[col] = df[col].astype(str)
woe.fit(df[features][:-len(test_df)], df['Attrition'][:-len(test_df)], cols = cat_features)
X = woe.transform(df[features])
X['Attrition'] = df['Attrition']
df = X

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# num_cols = ['age', 'avg_glucose_level', 'bmi']

y = df['Attrition']
df = df.drop(['id','Attrition'], axis=1)

df[df.columns] = scaler.fit_transform(df[df.columns])

In [ ]:
# y.shape

In [ ]:
train_df = df.iloc[:-len(test_df),:]
train_df['Attrition'] = y[:-len(test_df)]
test_df = df.iloc[-len(test_df):,:].reset_index(drop=True)

X = train_df.drop(['Attrition'], axis=1)
y = train_df.Attrition
X_test = test_df

In [ ]:
# X1 = X[:500]
# y1 = y[:500]
# X = X[500:]
# y = y[500:]

**CatBoost model**

In [ ]:
# from sklearn.model_selection import train_test_split
# import catboost
# from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
# from sklearn.metrics import roc_auc_score

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# from sklearn.model_selection import cross_val_score

# def catboost_objective(trial):
# #     learning_rate = trial.suggest_float('learning_rate', 0, 0.3)
# #     depth = trial.suggest_int('depth', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
#     subsample = trial.suggest_float('subsample', 0, 1)
#     l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 100)
#     min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 100)
    
#     model = catboost.CatBoostRegressor(
#         learning_rate= 0.2238051305181816,
#         depth= 3,
#         n_estimators= 500,
#         verbose= 0,
#         rsm =  0.5,
#         subsample = subsample,
#         l2_leaf_reg = l2_leaf_reg,
#         min_data_in_leaf= min_data_in_leaf,
#         random_strength = 0.175,
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(catboost_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
import catboost
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

n_folds = 11
repeats = 10

MAX_ITER = 15000
PATIENCE = 1000
DISPLAY_FREQ = 100

modelsCB = []
predsCB = []

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

MODEL_PARAMS = {
                'random_seed': 1234,             
                'iterations': MAX_ITER,
                'early_stopping_rounds': PATIENCE,
                'use_best_model': True,
#                 'eval_metric': 'AUC',
                'eval_metric': 'RMSE',
                'verbose': 1000,
    
                 'depth': 3, #3 2
                 'learning_rate': 0.2238051305181816, #0.2238051305181816, #0.01 0.45
                 'rsm': 0.5,
                 'subsample': 0.931, #0.1946411784581574, #0.931,
                 'l2_leaf_reg': 69, #39, #69, #69, 4.445
                 'min_data_in_leaf': 20, #74, #20, #20, 1
                 'random_strength': 0.175,
#                  'colsample_bylevel': 0.5504444444444444, #
               }


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = catboost.CatBoostClassifier(**MODEL_PARAMS)
    model = catboost.CatBoostRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
#           metric_period = DISPLAY_FREQ
         )
    modelsCB.append(model)
    predsCB.append(model.predict(X_test))
    #     eval_predsCB.append(model.predict(X))

In [ ]:
# from sklearn.model_selection import train_test_split
# import catboost
# from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
# from sklearn.metrics import roc_auc_score

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# from sklearn.model_selection import cross_val_score

# def catboost_cl_objective(trial):
# #     learning_rate = trial.suggest_float('learning_rate', 0, 0.3)
# #     depth = trial.suggest_int('depth', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
#     subsample = trial.suggest_float('subsample', 0, 1)
#     l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 100)
#     min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 100)
    
#     model = catboost.CatBoostClassifier(
#         learning_rate= 0.2238051305181816,
#         depth= 3,
#         n_estimators= 500,
#         verbose= 0,
#         rsm =  0.5,
#         subsample = subsample,
#         l2_leaf_reg = l2_leaf_reg,
#         min_data_in_leaf= min_data_in_leaf,
#         random_strength = 0.175,
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(catboost_cl_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
MAX_ITER = 15000
PATIENCE = 1000
DISPLAY_FREQ = 100

modelsCBC = []
predsCBC = []

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

MODEL_PARAMS = {
                'random_seed': 1234,             
                'iterations': MAX_ITER,
                'early_stopping_rounds': PATIENCE,
                'use_best_model': True,
                'eval_metric': 'AUC',
#                 'eval_metric': 'RMSE',
                'verbose': 1000,
    
                 'depth': 3, #3 2
                 'learning_rate': 0.2238051305181816, #0.08259958598203335, #0.2238051305181816, #0.01 0.45
                 'rsm': 0.5,
                 'subsample': 0.931, #0.008520274433054455, #0.931,
                 'l2_leaf_reg': 69, #62, #69, #69, 4.445
                 'min_data_in_leaf': 20, #69, #20, #20, 1
                 'random_strength': 0.175,
#                  'colsample_bylevel': 0.5504444444444444, #
               }


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = catboost.CatBoostClassifier(**MODEL_PARAMS)
#     model = catboost.CatBoostRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
#           metric_period = DISPLAY_FREQ
         )
    modelsCBC.append(model)
    predsCBC.append(model.predict_proba(X_test)[:, 1])


**XGBoost model**

In [ ]:
# def xgboost_objective(trial):
# #     learning_rate = trial.suggest_float('learning_rate', 0, 0.3)
# #     max_depth = trial.suggest_int('max_depth', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
#     subsample = trial.suggest_float('subsample', 0, 1)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
#     reg_lambda = trial.suggest_int('reg_lambda', 1, 100)
    
#     model = XGBRegressor(
#         learning_rate= 0.04625397031701272,
#         max_depth= 3,
#         n_estimators= 500,
#         colsample_bytree = colsample_bytree, # 0.95
#         subsample = subsample,
#         reg_lambda = reg_lambda,
# #         verbose= 0
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(xgboost_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsXB = []
predsXB = []

PATIENCE = 200

MODEL_PARAMS = {       'n_estimators': 5000, 
                       'learning_rate': 0.04625397031701272, #0.04625397031701272, 0.06733333333333334
                       'max_depth': 3, #3, 29
                       'colsample_bytree': 0.9, #0.12954517333371557, #0.9, #0.9, 0.99
                       'subsample': 1, #0.7426054009856451, #1, #1, 0.99
#                        'min_child_weight': 12, #
#                        'gamma': 0.11888888888888888, #
                       'reg_lambda': 20, #55, #20,
#                        'eval_metric': 'auc',
                       'eval_metric': 'rmse',
                       'early_stopping_rounds': PATIENCE,
#                        'tree_method': 'gpu_hist',
                       'seed': 1
}

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = XGBClassifier(**MODEL_PARAMS)
    model = XGBRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
          verbose = 100
         )
    modelsXB.append(model)
    predsXB.append(model.predict(X_test))

In [ ]:
# def xgboost_cl_objective(trial):
# #     learning_rate = trial.suggest_float('learning_rate', 0, 0.3)
# #     max_depth = trial.suggest_int('max_depth', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
#     subsample = trial.suggest_float('subsample', 0, 1)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
#     reg_lambda = trial.suggest_int('reg_lambda', 1, 100)
    
#     model = XGBClassifier(
#         learning_rate= 0.04625397031701272,
#         max_depth= 3,
#         n_estimators= 500,
#         colsample_bytree = colsample_bytree, # 0.95
#         subsample = subsample,
#         reg_lambda = reg_lambda,
#         eval_metric = 'auc',
# #         verbose= 0
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(xgboost_cl_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsXBC = []
predsXBC = []

PATIENCE = 200

MODEL_PARAMS = {       'n_estimators': 5000, 
                       'learning_rate': 0.04625397031701272, #0.02910875107189472, #0.04625397031701272, 0.06733333333333334
                       'max_depth': 3, #4, #3, 29
                       'colsample_bytree': 0.9, #0.11912735505392935, #0.9, #0.9, 0.99
                       'subsample': 1, #0.7605555695727094, # 1, #1, 0.99
#                        'min_child_weight': 12, #
#                        'gamma': 0.11888888888888888, #
                       'reg_lambda': 20, #17, #20,
                       'eval_metric': 'auc',
#                        'eval_metric': 'rmse',
                       'early_stopping_rounds': PATIENCE,
#                        'tree_method': 'gpu_hist',
                       'seed': 1
}

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = XGBClassifier(**MODEL_PARAMS)
#     model = XGBRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
          verbose = 100
         )
    modelsXBC.append(model)
    predsXBC.append(model.predict_proba(X_test)[:, 1])

**LGBM model**

In [ ]:
# def lgbm_objective(trial):
# #     learning_rate = trial.suggest_float('learning_rate', 0, 0.3)
# #     max_depth = trial.suggest_int('max_depth', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
#     num_leaves = trial.suggest_int('num_leaves', 1, 100)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
#     subsample = trial.suggest_float('subsample', 0, 1)
    
#     model = LGBMRegressor(
#         learning_rate= 0.04131536309750098,
#         max_depth= 3,
#         n_estimators= 500,
#         num_leaves = num_leaves,
#         colsample_bytree = colsample_bytree,
#         subsample = subsample,
#         subsample_freq = 5,
#         min_child_samples = 36,
#         reg_lambda = 28, 
#         metric = 'rmse',
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(lgbm_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=43, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsLB = []
predsLB = []

MODEL_PARAMS = {
                       'learning_rate': 0.04131536309750098,
                       'max_depth': 3,
                       'num_leaves': 90, #80, #90,
                       'colsample_bytree': 0.8, #0.1894464869106461, #0.8,
                       'subsample': 0.9, #0.5730986849967483, #0.9,
                       'subsample_freq': 5,
                       'min_child_samples': 36,
                       'reg_lambda': 28,
                       'n_estimators': 20000,
#                        'metric': 'auc',
                       'metric': 'rmse',
                       'random_state': 2
}

callbacks = [
             lgbm.early_stopping(30, verbose=1),
#              lgbm.log_evaluation(period=0)
            ]

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = lgbm.LGBMClassifier(**MODEL_PARAMS)
    model = lgbm.LGBMRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
          callbacks=callbacks
         )
    modelsLB.append(model)
    predsLB.append(model.predict(X_test))

In [ ]:
# def lgbm_cl_objective(trial):
#     learning_rate = trial.suggest_float('learning_rate', 0, 0.3)
#     max_depth = trial.suggest_int('max_depth', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
# #     num_leaves = trial.suggest_int('num_leaves', 1, 100)
# #     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
# #     subsample = trial.suggest_float('subsample', 0, 1)
    
#     model = LGBMClassifier(
#         learning_rate= learning_rate,
#         max_depth= max_depth,
#         n_estimators= 500,
#         num_leaves = 90,
#         colsample_bytree = 0.8,
#         subsample = 0.9,
#         subsample_freq = 5,
#         min_child_samples = 36,
#         reg_lambda = 28, 
#         metric = 'auc',
#         random_state = 2,
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(lgbm_cl_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsLBC = []
predsLBC = []

MODEL_PARAMS = {
                       'learning_rate': 0.04780445326920009, #0.04131536309750098,
                       'max_depth': 3,
                       'num_leaves': 90, #80, #90,
                       'colsample_bytree': 0.8, #0.1894464869106461, #0.8,
                       'subsample': 0.9, #0.5730986849967483, #0.9,
                       'subsample_freq': 5,
                       'min_child_samples': 36,
                       'reg_lambda': 28,
                       'n_estimators': 20000,
                       'metric': 'auc',
#                        'metric': 'rmse',
                       'random_state': 2
}

callbacks = [
             lgbm.early_stopping(30, verbose=1),
#              lgbm.log_evaluation(period=0)
            ]

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = lgbm.LGBMClassifier(**MODEL_PARAMS)
#     model = lgbm.LGBMRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
          callbacks=callbacks
         )
    modelsLBC.append(model)
    predsLBC.append(model.predict(X_test))

**Lasso regression**

In [ ]:
# def lasso_objective(trial):
#     eps = trial.suggest_float('eps', 0, 0.1)
#     cv = trial.suggest_int('cv', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
    
#     model = LassoCV(
#                        precompute = "auto",
#                        fit_intercept = True,
#                        normalize = False,
#                        max_iter = 1000,
#                        verbose = False,
#                        eps = eps,
#                        cv = cv,
#                        n_alphas = 1000,
#                        n_jobs = 8,         
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(lasso_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
from sklearn.linear_model import LassoCV

# n_folds = 20
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=2*repeats, random_state=42) # 20

modelsLR = []
predsLR = []

MODEL_PARAMS = {
                       'precompute': "auto",
                       'fit_intercept': True,
                       'normalize': False,
                       'max_iter': 1000,
                       'verbose': False,
                       'eps': 1e-04, #0.007267206407101401, #1e-04,
                       'cv': 5, #6, #5,
                       'n_alphas': 1000,
                       'n_jobs': 8,
}


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = LassoCV(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
#           eval_set=[(X_valid, y_valid)],
         )
    
    modelsLR.append(model)
    predsLR.append(model.predict(X_test))

In [ ]:
# def ridge_objective(trial):
#     eps = trial.suggest_float('eps', 0, 0.1)
#     cv = trial.suggest_int('cv', 3, 10)
# #     n_estimators = trial.suggest_int('n_estimators', 50, 500)
    
#     model = RidgeCV(
# #                        precompute = "auto",
# #                        fit_intercept = True,
# #                        normalize = False,
# #                        max_iter = 1000,
# #                        verbose = False,
# #                        eps = eps,
# #                        cv = cv,
# #                        n_alphas = 1000,
# #                        n_jobs = 8,         
#     )

#     kf = KFold(n_splits= 10)
#     cv_score = cross_val_score(model, X, y, scoring= 'roc_auc', cv= kf)
#     return np.mean(cv_score)

# study = optuna.create_study(direction= 'maximize')
# study.optimize(ridge_objective, n_trials= 50)

In [ ]:
# study.best_params

In [ ]:
from sklearn.linear_model import RidgeCV

# n_folds = 20
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=2*repeats, random_state=42) # 20

modelsR = []
predsR = []

# MODEL_PARAMS = {
#                        'precompute': "auto",
#                        'fit_intercept': True,
#                        'normalize': False,
#                        'max_iter': 1000,
#                        'verbose': False,
#                        'eps': 1e-04,
#                        'cv': 5,
#                        'n_alphas': 1000,
#                        'n_jobs': 8,
# }


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = RidgeCV(alphas=np.linspace(0.0001, 100, 1000)
                   )
    
    model.fit(X=X_train, y=y_train,
#           eval_set=[(X_valid, y_valid)],
         )
    
    modelsR.append(model)
    predsR.append(model.predict(X_test))

Submission from my notebook with Keras NN:
https://www.kaggle.com/code/alexandershumilin/ps-s3-e3-keras-nn

In [ ]:
# keras_nn_preds = pd.read_csv('/kaggle/input/keras-nn-submission-2/submission nn2.csv')
# predNN = keras_nn_preds.Attrition

In [ ]:
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import layers

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda, Concatenate, Add, BatchNormalization, LeakyReLU

from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold

from sklearn.metrics import classification_report

modelsNN = []
predsNN = []

class_weight = 10 

# n_folds = 7 #10
# repeats = 10 #10 
dr = 0.1     

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42)

def get_model():
    model = keras.Sequential([
    layers.Dense(512), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.Dense(256), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.Dense(128), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.Dense(64), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.BatchNormalization(),
    layers.Dense(32), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.Dense(16), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.Dense(8),
    layers.LeakyReLU(alpha=0.3),
    layers.Dropout(rate=dr),
    layers.Dense(4), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dense(2), 
    layers.LeakyReLU(alpha=0.3),
    layers.Dense(1, activation='sigmoid')
   ])

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(
    optimizer=opt,
        loss=tfa.losses.SigmoidFocalCrossEntropy(
                                             alpha=0.8,
                                             gamma=2.0
                                             ),
        metrics='AUC',
)
    
    return model

early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_auc", 
        mode='max',
        patience=30,
        min_delta=0.00001,
        restore_best_weights=True,
)
plat = keras.callbacks.ReduceLROnPlateau(
        monitor="val_auc", 
        mode='max', 
        patience=3, 
        factor=0.1, 
        min_lr=1e-8, 
        min_delta=0.000001)


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = get_model()

    history = model.fit(
          X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=64,
          epochs=500,
          class_weight = { 0: 1.0, 1: class_weight, },
          callbacks=[early_stopping, plat],
          verbose=0
         )
    
#     print(classification_report(y, np.round(model.predict(X))))
    modelsNN.append(model)
    predsNN.append(model.predict(X_test))

In [ ]:
from sklearn.metrics import roc_auc_score

 
def coef_objective(trial):
    a = trial.suggest_float('a', 0, 1)
    b = trial.suggest_float('b', 0, 1)
    c = trial.suggest_float('c', 0, 1)
    d = trial.suggest_float('d', 0, 1)
    e = trial.suggest_float('e', 0, 1)
    f = trial.suggest_float('f', 0, 1)
    g = trial.suggest_float('g', 0, 1)
    h = trial.suggest_float('h', 0, 1)
    i = trial.suggest_float('i', 0, 1)
    
#     X = X1
#     y = y1
    
    preds_eval = []
    for model in modelsCB:
        preds_eval.append(model.predict(X))
    
    resCB = np.average(np.array(preds_eval),axis=0)

    preds_eval = []
    for model in modelsXB:
        preds_eval.append(model.predict(X))
    
    resXB = np.average(np.array(preds_eval),axis=0)

    preds_eval = []
    for model in modelsLB:
        preds_eval.append(model.predict(X))
    
    resLB = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsLR:
        preds_eval.append(model.predict(X))
    
    resLR = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsCBC:
        preds_eval.append(model.predict_proba(X)[:, 1])
    
    resCBC = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsXBC:
        preds_eval.append(model.predict_proba(X)[:, 1])
    
    resXBC = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsR:
        preds_eval.append(model.predict(X))
    
    resR = np.average(np.array(preds_eval),axis=0)
    
#     k = pd.read_csv('/kaggle/input/keras-nn-test/KerasNNX1.csv')
#     resNN = np.array(k['0'])
    preds_eval = []
    for model in modelsNN:
        preds_eval.append(model.predict(X))
    
    resNN = np.average(np.array(preds_eval),axis=0).T[0]
    
    preds_eval = []
    for model in modelsLBC:
        preds_eval.append(model.predict(X))
    
    resLBC = np.average(np.array(preds_eval),axis=0)

    res = roc_auc_score(y,
                        (resCB * a + resXB * b + resLB * c + resLR * d + resNN * e + resCBC * f + resXBC * g + resR * h + resLBC * i)/(a + b + c + d + e + f + g + h + i))

    return res

study = optuna.create_study(direction= 'maximize')
# study.optimize(coef_objective, n_trials= 500)

In [ ]:
# study.best_params

In [ ]:
# a = study.best_params['a']
# b = study.best_params['b']
# c = study.best_params['c']
# d = study.best_params['d']
# e = study.best_params['e']
# f = study.best_params['f']
# g = study.best_params['g']
# h = study.best_params['h']
# i = study.best_params['i']
# sum_coef = a + b + c + d + e + f + g + h + i
# a = a / sum_coef
# b = b / sum_coef
# c = c / sum_coef
# d = d / sum_coef
# e = e / sum_coef
# f = f / sum_coef
# g = g / sum_coef
# h = h / sum_coef
# i = i / sum_coef
# a, b, c, d, e, f, g, h, i

In [ ]:
# a = 0.2 #0.175 < #0.2 > #0.25 #0.27
# b = 0.2 #0.175 < #0.2 > #0.25 #0.27
# c = 0.2 #0.175 < #0.2 > #0.25 #0.26
# d = 0.2 #0.175 < #0.2 > #0.25 #0.2
# # e = 0.2
# f = 0.07 #0.1 < #0.07 >
# g = 0.07 #0.1 < #0.07 >
# h = 0.06 #0.1 < #0.06 >

In [ ]:
# a = 0.09
# b = 0.03
# c = 0.01
# d = 0.34
# f = 0.14
# g = 0.06
# h = 0.33

In [ ]:
# a = 0.03065532725826637
# b = 0.011578190971283535
# c = 0.019159393835472383
# d = 0.3373901994395983
# e = 0.03422756360477424
# f = 0.21937795907658797
# g = 0.00035405910179546316
# h = 0.3472573067122217

In [ ]:
# a = 0.021187836414304755
# b = 0.014267651652556123
# c = 0.00020261132653446776
# d = 0.2681613387688626
# e = 0.11695188957003269
# f = 0.2115865371220429
# g = 0.014728457554323478
# h = 0.23683829798854342
# i = 0.11607537960279952

In [ ]:
a = 0.23061901493810577
b = 0.10878749005253482
c = 0.010220344157544843
d = 0.21876888201581107
e = 0.02803722686774301
f = 0.006502031954055989
g = 0.006070445735349712
h = 0.16421910663179387
i = 0.22677545764706097

In [ ]:
# a = 0.06835812076695698
# b = 0.009738413349831195
# c = 0.0006182592059716698
# d = 0.36072715934078853
# e = 0.007755560801158053
# f = 0.03392360968321388
# g = 0.03714379953405301
# h = 0.27245537702078987
# i = 0.20927970029723697

In [ ]:
predCB = np.average(np.array(predsCB),axis=0).clip(0,1)
predXB = np.average(np.array(predsXB),axis=0).clip(0,1)
predLB = np.average(np.array(predsLB),axis=0).clip(0,1)
predLR = np.average(np.array(predsLR),axis=0).clip(0,1)
predCBC = np.average(np.array(predsCBC),axis=0).clip(0,1)
predXBC = np.average(np.array(predsXBC),axis=0).clip(0,1)
predR = np.average(np.array(predsR),axis=0).clip(0,1)
predLBC = np.average(np.array(predsLBC),axis=0).clip(0,1)
predNN = np.average(np.array(predsNN),axis=0).T[0].clip(0,1)

pred = predCB * a + predXB * b + predLB * c + predLR * d  + predNN * e + predCBC * f + predXBC * g + predR * h  +  predLBC * i

pred

In [ ]:
pd.Series(predCB).hist()

In [ ]:
pd.Series(predXB).hist()

In [ ]:
pd.Series(predLB).hist()

In [ ]:
pd.Series(predLR).hist()

In [ ]:
pd.Series(predNN).hist()

In [ ]:
pd.Series(predCBC).hist()

In [ ]:
pd.Series(predXBC).hist()

In [ ]:
pd.Series(predR).hist()

In [ ]:
pd.Series(predLBC).hist()

In [ ]:
pd.Series(pred).hist()

**Making submission**

In [ ]:
submission['Attrition'] = pred
submission

In [ ]:
submission.to_csv('submission.csv', index=False)